In [1]:
import requests
from bs4 import BeautifulSoup
url = 'https://www.airlinequality.com/airline-reviews/british-airways'

In [24]:

reviews = {'name': [],'country': [], 'date': [],
    #            'trip_verified': trip_verified,
    #            'rating':rating,
    #            'aircraft':aircraft,
    #            'type_of_traveller': type_of_traveller,
    #            'seat_type': seat_type,
    #            'route': route,
    #            'date_flown': date_flown,
    #            'seat_comfort': seat_comfort,
    #            'cabin_staff_service': cabin_staff_service,
    #            'food_beverages': food_beverages,
    #            'entertainment': entertainment,
    #            'ground_service': ground_service,
    #            'wifi': wifi,
    #            'value_for_money': value_for_money,
    #            'recommended': recommended
            }

review_keys = ['name', 'country', 'date']

def find_country(single_review):
    h3_tag = single_review.find('h3', class_='text_sub_header userStatusWrapper')
    if h3_tag:
        country_element = h3_tag.text
        value = country_element.split('(')[-1].split(')')[0].strip()
    else:
        value = None

    reviews.setdefault('country', []).append(value)

def find_itemprop(single_review, variable=None, attribute=None, get_value=None):
  if attribute and attribute is not None:
    element = single_review.find(attribute, {'itemprop': variable})
  elif attribute is None:
    element = single_review.find(variable).get(get_value)



  if element:
      if variable == 'time':
        reviews.setdefault('date', []).append(element)

      else:
        value = element.get_text(strip=True)
        if variable == 'name':
          reviews.setdefault('name', []).append(value)
        elif variable == 'country':
          reviews.setdefault('country', []).append(value)
        elif variable == 'datePublished':
          reviews.setdefault('date', []).append(value)
  else:
      reviews.setdefault('No column in itemprop', []).append(attribute)

def scrape_reviews(current_page_url):
  # Send a GET request to the URL
  response = requests.get(current_page_url)
  # Check if the request was successful (status code 200)
  if response.status_code == 200:
      # Parse the HTML content of the page using BeautifulSoup
      soup = BeautifulSoup(response.text, 'html.parser')

      # Find all review elements on the page
      all_reviews = soup.find_all('article', {'itemprop': 'review'})

      for idx, single_review in enumerate(all_reviews, start=1):
          variable_attributes = {
              'name': 'span',
              'country': 'span',
              'time': 'datetime'
          }

          for variable, attribute in variable_attributes.items():
              if variable == 'name':
                find_itemprop(single_review, variable, attribute)
              elif variable == 'country':
                find_country(single_review)
              elif variable == 'time':
                find_itemprop(single_review, variable, get_value=attribute)


  else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


num_pages = 2

# Iterate through the pages
for page_number in range(1, num_pages + 1):
  # Generate the URL for the current page
  current_page_url = f"{url}/page/{page_number}/"
  print(current_page_url)
  # Call the function to scrape reviews from the current page
  scrape_reviews(current_page_url)


# Now 'reviews' contains lists of names and countries in the correct order
for key, values in reviews.items():
  print(f'{key}:')
  for value in values:
    print(f'    {value}')
print()  # Print an empty line between key-value pairs


https://www.airlinequality.com/airline-reviews/british-airways/page/1/
https://www.airlinequality.com/airline-reviews/british-airways/page/2/
name:
    A Gittens
    Bert Vissers
    E Smyth
    L Keele
    Austin Jones
    M A Collie
    Nigel Dean
    Gaylynne Simpson
    A Narden
    Graeme Boothman
    R Vines
    Massimo Tricca
    J Kaye
    M Collie
    J Furlong
    C Barton
    M Davidson
    Michael Gardiner
    Terry Anderson
    Philip Thompson
country:
    United Kingdom
    Switzerland
    United Kingdom
    Germany
    United States
    Ireland
    Australia
    United States
    Romania
    United Kingdom
    United Kingdom
    Italy
    United Kingdom
    Ireland
    United States
    United Kingdom
    United States
    United Kingdom
    United States
    United Kingdom
date:
    2023-11-30
    2023-11-26
    2023-11-23
    2023-11-19
    2023-11-19
    2023-11-16
    2023-11-16
    2023-11-14
    2023-11-12
    2023-11-08
    2023-11-07
    2023-11-05
    2023-11-05